# News Search and Summarizer

This notebook searches for latest news and provides summaries.

In [ ]:
# Install required packages (run once)
#%pip install openai feedparser requests python-dotenv

In [ ]:
# import feedparser
# import requests
# from openai import OpenAI
# from datetime import datetime

# %pip install  dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dotenv]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import feedparser
import requests
from openai import OpenAI
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
# Initialize OpenAI client
# API key is loaded from .env file
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file. Please add it to your .env file.")

client = OpenAI(api_key=api_key)

## 1. Search Latest News

Using Google News RSS feed (free, no API key needed)

In [3]:
def fetch_latest_news(topic='world', num_articles=5):
    """
    Fetch latest news from Google News RSS feed
    
    Args:
        topic: News topic (e.g., 'world', 'technology', 'business')
        num_articles: Number of articles to fetch
    """
    url = f'https://news.google.com/rss/search?q={topic}&hl=en-US&gl=US&ceid=US:en'
    
    feed = feedparser.parse(url)
    
    articles = []
    for entry in feed.entries[:num_articles]:
        article = {
            'title': entry.title,
            'link': entry.link,
            'published': entry.published if 'published' in entry else 'N/A',
            'summary': entry.summary if 'summary' in entry else entry.title
        }
        articles.append(article)
    
    return articles

In [4]:
# Fetch latest news
topic = 'technology'  # Change to any topic you want
news_articles = fetch_latest_news(topic=topic, num_articles=5)

print(f"Latest {topic} news:\n")
for i, article in enumerate(news_articles, 1):
    print(f"{i}. {article['title']}")
    print(f"   Published: {article['published']}")
    print(f"   Link: {article['link']}")
    print()

Latest technology news:

1. Carney signs onto AI and technology partnership with India and Australia - CBC
   Published: Sat, 22 Nov 2025 15:34:53 GMT
   Link: https://news.google.com/rss/articles/CBMihwFBVV95cUxNeUNTMkhOc2JibXRiRkRWXzJvU2JnaF9sM2tZSUlzM0dLQnNidmNoNE02M3pmX0JJYXRjNzE4QXdGNnZ3M1lLSGNDZDJ3ZE1CWkpFRXhsX3FfTTFvVFNmVmpwTXpycndyUTlkNnZTN09MaXpwQlNYaV9uVzJGZ3I4UjJWZzBhV3c?oc=5

2. PM Modi meets Mark Carney, Albanese; announces Australia-Canada-India Technology & Innovation Partnership - livemint.com
   Published: Sat, 22 Nov 2025 12:59:07 GMT
   Link: https://news.google.com/rss/articles/CBMi-AFBVV95cUxQMzBaTkZ0Wjg3ZGt4cUxVazVRNlQ5Qktua1daXzJpaWxxYWN4WGZOZHRubXZrWXZqWmdxX21LUm9aRzVwdDE3TmE2R1BOUnE4a2tmT21hQVFpZkp4bVhQNXlyVEloMzNGVl8tU1JiTjl6ZWtxUTNPS1NsbHJMUExVaHFiTnhNYjY4aHVLTTRDLWJBRHRGODl6Rmc3Q2pPWmt3SUJKVldPVVJDTVRBTmd1WUkyMnlUZ1BJbGZIVUswTDFBa3dkZllQYVFQbHpvdWJ2OVBLRWYwZ2FqTklWWUtVdG1mY3RyTmZ4cGRDeGlUTjJwbGxEV1BLVdIB_gFBVV95cUxPUGNWLUxmeFZGVmZLWmNoNldLd2NLNmxPaklxNjdNbHB

## 2. Summarize News with OpenAI

In [6]:
def summarize_news_with_openai(articles):
    """
    Create a summary of multiple news articles using OpenAI
    """
    # Combine all article titles and summaries
    news_text = "\n\n".join([
        f"Article {i+1}: {article['title']}\n{article['summary']}"
        for i, article in enumerate(articles)
    ])
    
    # Call OpenAI API
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Use gpt-4o or gpt-3.5-turbo based on your preference
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes news articles concisely."},
            {"role": "user", "content": f"Please provide a brief summary of these news articles:\n\n{news_text}"}
        ],
        max_tokens=500,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [7]:
# Generate summary
summary = summarize_news_with_openai(news_articles)

print("="*60)
print("NEWS SUMMARY")
print("="*60)
print(summary)
print("="*60)

NEWS SUMMARY
**Article 1:** Mark Carney, the Prime Minister of Canada, has joined a new AI and technology partnership with India and Australia. The initiative aims to foster collaboration in technological advancements among the three nations.

**Article 2:** During a meeting with PM Modi of India and PM Albanese of Australia at the G20 Summit, Mark Carney announced the establishment of the Australia-Canada-India Technology and Innovation Partnership, which seeks to enhance cooperation in technology and innovation sectors.

**Article 3:** Prime Minister Modi expressed enthusiasm for the new trilateral technology and innovation partnership formed with Mark Carney and Anthony Albanese, highlighting the significance of their discussions on the sidelines of the G20 Summit in Johannesburg.

**Article 4:** A recent awards ceremony recognizing engineers has underscored China's advancements in military technology, showcasing the country's growing capabilities in this sector.

**Article 5:** The

## 3. All-in-One Function

In [18]:
def search_and_summarize_news(topic='world', num_articles=5):
    """
    Search for news and generate a summary in one function
    """
    print(f"Fetching latest news about '{topic}'...\n")
    articles = fetch_latest_news(topic=topic, num_articles=num_articles)
    
    print(f"Found {len(articles)} articles:\n")
    for i, article in enumerate(articles, 1):
        print(f"{i}. {article['title']}")
    
    print("\nGenerating summary...\n")
    summary = summarize_news_with_openai(articles)
    
    print("="*60)
    print("SUMMARY")
    print("="*60)
    print(summary)
    print("="*60)
    
    return articles, summary

In [20]:
# Quick usage example
#articles, summary = search_and_summarize_news(topic='artificial intelligence', num_articles=5)